Dou et al. (2020) model

Agora em Julia versão 1.5.3

This script simulates the model

Calcula momentos a partir de parâmetros arbitrários a fim de testar o algoritmo de minimização.

In [1]:
using LinearAlgebra, Statistics

using DataFrames
using BenchmarkTools
using DelimitedFiles

In [2]:
using Distributed
nprocs = 2
addprocs(nprocs);

# @everywhere using NBInclude
# @everywhere @nbinclude("solve_model.ipynb")

#importou a função solve_tree

pids = workers()
aux = pids[nprocs]

3

In [3]:
@everywhere function solve_tree(data, game_parameters, grid=100, hlow=0.01, hhigh=1.0, grid_payment=1000)
    
#     #data ####
#     #all the data is escalated so as to D = 1.0
    Vmax = data[1]; #firm's maximum reorganization value
    L = data[2]; #firm's maximum liquidation value
    Dj = data[3]; #junior's debt face value
    Ds = 1.0 - Dj; #senior's debt face value

    D = Ds + Dj; #total debt, used to escale costs

#     #parameters####
#     #parameters include only the parameters necessary to calculate the model, so it does not include θs0, θj0, μ
    ρ = game_parameters[1]; # (1 - ρ) is the firm's depreciation rate each period
    β = game_parameters[2]; #inverse of the speed of learning
    c0 = game_parameters[3]; #fixed cost of going to court
    λj = game_parameters[4]; #chance of j being called to propose at each period

    #c1 is calibrated. We will start with the same c1 as Dou et al.(2019)
    c1 = c0/30; #variable cost for each period at court
    #Initial definitions: costs, maximum value of the firm, maximum number of periods####


    #number of periods
    function max_turns(Vmax, L, ρ, c0, c1)
        t = 0

        #using the specification of the theoretical model
        #we are not considering the change in indexation from t to t+1 yet

        #the condition L - c0 - c1*t >= 0 assures that the liquidation payoffs won't be negative

        while( (ρ^(t-1) * Vmax > L) && (L - c0 - c1*t >= 0) )
            t = t+1
        end
        return t

    end


    T = max_turns(Vmax, L, ρ, c0, c1)


    #cost function
    function Ct(t, c0=c0, c1=c1)
        #cost at period t=0(index1) is 0
        if(t <= 1)
            return 0
        else
            return c0 * D + c1 * (t-1) * D #test to make index==1 be t==0
        end
    end

    #maximum value of reorganization each period
    function Vt(Vmax, ρ, t)

        if(t <=1)
            return Vmax
        else
            #(t-2) instead of (t-1) because we shifted the indexes in the game so as to include t==0 at index==1
            return ρ^(t-2) * Vmax
        end


    end
    #index 1 corresponds to t=0 and 2 to t=1...

    #liquidation payoffs

    function s_L(t)
        return min(L - Ct(t), Ds)
    end

    function j_L(t)
        return min(L - Ct(t) - s_L(t), Dj)
    end



    #grid size
    grid = grid

    hlow = hlow
    hhigh = hhigh


    hvals = LinRange(hlow, hhigh, grid)



    #array to tell the size of the pie at period t, according to reorganization skill levels
    #arguments: (period, θkt)
    U = zeros(T+1, grid)


    for t in 1:T+1
        for h in 1:grid
            U[t,h] = hvals[h] * Vt(Vmax, ρ, t) - Ct(t)
        end
    end

    #cumulative distribution function
    function cdf(x, lt, β=β)

        #return the cdf of x given the lower bound lt
        #geq than 1 because of our discretization method 
        if(x >= 1.0)

            return 1.0

        else

            if(x >= lt)
                return 1.0 - ((1.0-x)^β)/((1-lt)^β)
            else
                return 0.0
            end
        end
    end


    #pmf: each point of the grid is at the center of the cdf
    function pmf_cdf(lt, δ = hlow/2, β=β)
        pmf = zeros(grid)

        for (i, h) in enumerate(hvals)
            pmf[i] = cdf(h+δ, lt) - cdf(h-δ, lt)
        end

        return pmf
    end



    #generating the pmfs

    #probability mass function
    pmf = zeros(grid,grid)

    δ = hlow/2
    for (i, htoday) in enumerate(hvals)
        for (j, htomorrow) in enumerate(hvals)
            pmf[i,j] = cdf(htomorrow+δ, htoday) - cdf(htomorrow-δ, htoday)
        end
    end
    


    #we need a pmf for when we have lkt and want to know θk,t+1
    #we will call it 'pmf2'
    pmf2 = zeros(grid,grid)

    for i in eachindex(hvals)
        for j in eachindex(hvals)
            pmf2[i, j] = sum(pmf[i, :] .* pmf[:, j])
        end
    end



    #continuation values
    #period t (not t+1), θkt, ℓkt, ℓmt
    s_W = zeros(T+1, grid, grid, grid);
    j_W = zeros(T+1, grid, grid, grid);


    #optimal payments
    #period, θkt, ℓmt, outputs
    Pst_array = zeros(T, grid, grid, 6); #we won't need payment proposal for period T, a.k.a. "T+1"
    Pjt_array = zeros(T, grid, grid, 6);

    #populating the last period with the liquidation values
    s_W[T+1, :, :, :] .= s_L(T+1); #we need to use .= because it is a broadcast
    j_W[T+1, :, :, :] .= j_L(T+1);

    #function to select the pmf function as it is writen in the paper

    function populate_probm!(t, lmt, probm, pmf, pmf2, grid)
        #takes the indexes of cutoff and lmt

        #at t=0(index 1), all information is symmetric, so we use pmf instead of pmf2
        if(t <= 1)

            for i in 1:grid
                probm[i] = pmf[lmt, i]
            end

        else

            for i in 1:grid
                probm[i] = pmf2[lmt, i]
            end

        end
    end



    #cutoff function: upgrade

    function getcutoff_manual(t, pkt, hkt, m_W, grid)


        cmt=Int64(1)

        while(m_W[t+1, cmt, cmt, hkt] < pkt)
            cmt+=1

            #break if cmt==grid+1
            if(cmt==grid+1)
                return cmt
            end

        end

        return cmt
    end



    #function to "filter" the cutoff

    function cutoff_manual(t, pkt, hkt, lmt, m_W, grid)
        #for now, it will just tell us if the payment offer is negative

        #if payment is negative (waiting offer), the cutoff will be 102 (grid+2), a code for the adversary to always reject it
        if(pkt<0)
             return grid+2
        end

        cmt = getcutoff_manual(t, pkt, hkt, m_W, grid)

        return cmt


    end


    function dot_W(t, hkt, lkt, lmt, grid, k_W, pmf)

        #calculates the dot product between k_W and pmf

        payoff=0.0
        @inbounds for i in hkt:grid
            payoff+= k_W[t+1, i, lkt, lmt] * pmf[hkt, i]
        end
        return payoff
    end

    #lkt+1 = hkt, so we input it here

    function dot_U(t, hkt, grid, U, pkt, pmf)

        #calculates the dot product between U and pmf
        payoff=0.0
        @inbounds for i in hkt:grid
            payoff+= (U[t, i] - pkt) * pmf[hkt, i]
        end
        return payoff
    end
    
    
    function lastindmax(x)
        #findmax, but returns higher index when two elements are equal
       k = 1
       m = x[1]
       @inbounds for i in eachindex(x)
           if x[i]>=m
               k = i
               m = x[i]
           end
       end
       return k
    end

    function compute_payoff_pkt2!(t, payoff, pmf, probm, k_W, m_W, U, pkt_grid, hkt, lmt, Vmax, grid)
    
        @inbounds for i in 1:(length(pkt_grid))

            pkt = pkt_grid[i]
            cmt = lmt + (i)



            if(pkt<0)
                cmt = grid+2
            end

            if(cmt==grid+2) 
                #means that pkt<0, so it will be always rejected
                #lmt+1 == lmt
                payoff[i] += dot_W(t, hkt, hkt, lmt, grid, k_W, pmf) * sum((probm))
    #             function dot_W(t, hkt, lkt, lmt, grid, k_W, pmf)

    #                 #calculates the dot product between k_W and pmf

    #                 payoff=0.0
    #                 @inbounds for i in hkt:grid
    #                     payoff+= k_W[t+1, i, lkt, lmt] * pmf[hkt, i]
    #                 end
    #                 return payoff
    #             end



            elseif(cmt == grid+1) 
                #means that cutoff doesn't exist, so the payment offer will be accepted by all θmt+1

                #since sum(probm[lmt:grid])==1, we didn't include it here
                payoff[i] += dot_U(t, hkt, grid, U, pkt, pmf) * sum((probm))
    #             function dot_U(t, hkt, grid, U, pkt, pmf)

    #                 #calculates the dot product between U and pmf
    #                 payoff=0.0
    #                 @inbounds for i in hkt:grid
    #                     payoff+= (U[t, i] - pkt) * pmf[hkt, i]
    #                 end
    #                 return payoff
    #             end

            else

                #cmt > lmt
                    #accepted if hmt+1 <=cmt
                #se a habilidade é igual ao cutoff então o adversário rejeita
                    payoff[i] += dot_U(t, hkt, grid, U, pkt, pmf) * sum(@view(probm[lmt:(cmt)]))


                    #rejected otherwise
                    #lmt+1 = cmt
                    payoff[i] += dot_W(t, hkt, hkt, cmt, grid, k_W, pmf) * sum(@view(probm[(cmt+1):grid]))

            end
        end
    end





    grid_payment=grid_payment
    share = LinRange(hlow, hhigh-hlow, grid_payment)

    #testing cutoff in this newer version of Pkt

    #function to calculate the optimal payment offer and directly populate the array

    function Pkt!(t, hkt, lmt, k_W, m_W, k_L, Pkt_array, pmf, pmf2, share, U, Vmax=Vmax, grid=grid)
    
        #pmf of m is a function of t (at t=0 (index 1), the information is symmetric)
        #array: pmf[lmt, 1:end] or pmf2[lmt, 1:end]
        probm = zeros(grid)
        populate_probm!(t, lmt, probm, pmf, pmf2, grid)

        pkt_grid = zeros(size(lmt:grid,1))

        min_share = 0.0001

        #popula o grid de pagamentos de acordo com os valores de continuação do adversário
        for (i, cmt) in enumerate(lmt+1:grid)

            share = m_W[t+1, cmt, cmt, hkt]

            if(U[t,hkt] <= share) #to avoid payment promisses that won't be fulfilled
                pkt_grid[i] = -Vmax

            else
                pkt_grid[i] = share

            end


             if(pkt_grid[i] == zero(1))
                    pkt_grid[i] = U[t, hkt] * min_share
                end
        end

        pkt_grid[size(lmt:grid,1)] = m_W[t+1, grid, grid, hkt] + U[t, hkt] * min_share

        payoff = zeros(length(pkt_grid))
        compute_payoff_pkt2!(t, payoff, pmf, probm, k_W, m_W, U, pkt_grid, hkt, lmt, Vmax, grid)

        #payoffs matrix####



        index_reorg = lastindmax(payoff)
        payoff_reorg = payoff[index_reorg]
        payment = pkt_grid[index_reorg] 
        cmt = lmt + (index_reorg) #tem que seguir a mesma regra do cmt de dentro de compute_payoff_pkt!
        if(payment <0)
            cmt = grid+2
        end

    #     if(index_reorg==1)

    #         payment = -Vmax
    #         cmt = grid+2

    #     else
    #         payment = pkt_grid[index_reorg-1]
    #         cmt = lmt + index_reorg - 1
    #     end    



        #calculating the optimal policy between liquidating, reorganizing or waiting ####
        payoff_liq = k_L(t)

        #waiting payoff is associated with the first possible payment, the waiting offer(-Vmax)
        payoff_wait = dot_W(t, hkt, hkt, lmt, grid, k_W, pmf) * sum(probm)

        #se o valor da torta for negativo, os cálculos de reorganização ficam errados
        #se o valor da torta for negativo, o jogador ou espera ou liquida a firma
        if(U[t,hkt] <= 0.0)
            payoff_reorg = payoff_wait
            cmt = grid+2
            payment = -Vmax
        end

        payoff_max, policy = findmax((payoff_liq, payoff_wait, payoff_reorg))

        if(payoff_max == payoff_wait)
            cmt = grid+2
            payment = -Vmax
        end



        #populates the array
        Pkt_array[t, hkt, lmt, :] .= payment, cmt, payoff_reorg, payoff_wait, payoff_max, policy

    end





    #optimal proposal
    function propose(t, hkt, lkt, lmt, Pkt_array)


        #it just searches for the corresponding Pkt
        payoff_max, policy = @views Pkt_array[t, hkt, lmt, end-1:end]

        return payoff_max, policy
    end


    #"cdf" for the discretized distribution
    function cdf_discrete(x, lt, pmf)

        #returns the probability that θt takes a value less than or equal to x, given lt
        #if needed, ie in threshold, we can just subtract 0.01 from x, so cdf_discrete the prob that θt < x, strictly
        return sum(@view(pmf[lt, lt:x]))
    end



    #function to calculate the payment offer threshold ϕmt

    function threshold_m(t, lkt, lmt, Pmt_array)

        #searches the index where the optimal policy is either 2.0 or 3.0 (wait or reorganize)
        #"Return the index of the first value in a greater than or equal to x, according to the specified order. 
        #Return length(a) + 1 if x is greater than all values in a. a is assumed to be sorted."
        ts = @views searchsortedfirst(Pmt_array[t, :, lkt, end], 2.0)


        #returns error message if threshold does not exist (101 is returned if missing by searchsortedfirst)
#         @assert ts < 101 "ρ=$ρ, c0=$c0"
        #removed @assert because it is not necessary to calculate the equilibrium of the game

        return ts

    end


    function prob_liq_m(t, ts, lmt, pmf)

        #probability that m will propose liquidation next period

        #at t==0 (index 1), the info is symmetric, so lmt == θmt, thus it becomes the indicator function
        if(t<=1)
            if(lmt >= ts)
                return 0.0
            else
                return 1.0
            end


        else#if t>0(index>1)

            if(lmt >= ts)
                return 0.0
            else
                return cdf_discrete(ts-1,lmt, pmf) #we subtract 1 because we want prob(θt < ts)
            end

        end


    end


    function dot_W2(t, hkt, hkt_next, lkt, lmt, grid, k_W, pmf)

        #calculates the dot product between k_W and pmf
        #modified to include hkt_next, to use in respond_payment

        payoff=0.0
        @inbounds for i in hkt_next:grid
            payoff+= k_W[t+1, i, lkt, lmt] * pmf[hkt, i]
        end
        return payoff
    end

    # function compute_payoff2!(t, payoff, Pkt_grid, cmt, pmf, probm, k_W, m_W, U, hkt, lmt, grid=grid)

    function compute_payoff_pmt(t, Pmt_array, pmf, k_W, hkt, lkt, lmt, grid=grid)
        payoff=zero(0.0)


        @inbounds for lmt_next in lmt:grid
            pmt = Pmt_array[t, lmt_next, lkt, 1]
            ckt = Int64(Pmt_array[t, lmt_next, lkt, 2])

            if(ckt==grid+2)
                #means that the payment offer is negative (waiting offer)
                #the proposal will be rejected for sure, without lowerbound update
    #             payoff += dot(@view(k_W[t+1, hkt:end, lkt, lmt_next]), @view(pmf[hkt, hkt:end])) * pmf[lmt, lmt_next]
                payoff += dot_W2(t, hkt, hkt, lkt, lmt_next, grid, k_W, pmf) * pmf[lmt, lmt_next]


            elseif(ckt==grid+1)
                #means that there is no cutoff, K will accept the payment offer for all θkt+1
    #             payoff += pmt * sum(@view(pmf[hkt, hkt:end])) * pmf[lmt, lmt_next]
                payoff += pmt * sum(@view(pmf[hkt, hkt:grid])) * pmf[lmt, lmt_next] 

            else

                if(ckt < lkt)
                    #the proposal will be rejected for sure, without lowerbound update
    #                 payoff += dot(@view(k_W[t+1, hkt:end, lkt, lmt_next]), @view(pmf[hkt, hkt:end])) * pmf[lmt, lmt_next]

                    payoff += dot_W2(t, hkt, hkt, lkt, lmt_next, grid, k_W, pmf) * pmf[lmt, lmt_next]


                elseif(ckt==lkt)
                    #will accept only if hkt+1==lkt
                    payoff += pmt * pmf[hkt, lkt] * pmf[lmt,lmt_next]

                    #rejects otherwise
    #                 payoff += dot(@view(k_W[t+1, lkt+1:end, lkt, lmt_next]), @view(pmf[hkt, lkt+1:end])) * pmf[lmt, lmt_next]

                    payoff+= dot_W2(t, hkt, lkt+1, lkt, lmt_next, grid, k_W, pmf) * pmf[lmt, lmt_next]

    #                 function dot_W2(t, hkt, hkt_next, lkt, lmt, grid, k_W, pmf)

    #                     #calculates the dot product between k_W and pmf
    #                     #modified to include hkt_next, to use in respond_payment

    #                     payoff=0.0
    #                     @inbounds for i in hkt_next:grid
    #                         payoff+= k_W[t+1, i, lkt, lmt] * pmf[hkt, i]
    #                     end
    #                     return payoff
    #                 end


                else
                #now ckt > lkt

                    if(ckt < hkt)
                        #he rejects, but there is an update in the lowerbound lkt+1==ckt
    #                     payoff += dot(@view(k_W[t+1, hkt:end, ckt, lmt_next]), @view(pmf[hkt, hkt:end])) * pmf[lmt, lmt_next]

                        payoff += dot_W2(t, hkt, hkt, ckt, lmt_next, grid, k_W, pmf) * pmf[lmt, lmt_next]

                    else

                        #receives pmt if his skill level is below or equal to the cutoff
    #                     payoff += pmt * sum(@view(pmf[hkt, hkt:ckt])) * pmf[lmt, lmt_next]

                        payoff += pmt * sum(@view(pmf[hkt, hkt:(ckt)])) * pmf[lmt, lmt_next]


                        #receives continuation value k_W if his skill level greather than the cutoff
    #                     payoff += dot(@view(k_W[t+1, ckt+1:end , ckt, lmt_next]), @view(pmf[hkt, ckt+1:end])) * pmf[lmt, lmt_next]


    #                     payoff+= dot_W2(t, hkt, ckt+1, ckt, lmt_next, grid, k_W, pmf) * pmf[lmt, lmt_next]

                        payoff+= dot_W2(t, hkt, ckt+1, ckt, lmt_next, grid, k_W, pmf) * pmf[lmt, lmt_next]


    #                     function dot_W2(t, hkt, hkt_next, lkt, lmt, grid, k_W, pmf)

    #                         #calculates the dot product between k_W and pmf
    #                         #modified to include hkt_next, to use in respond_payment

    #                         payoff=0.0
    #                         @inbounds for i in hkt_next:grid
    #                             payoff+= k_W[t+1, i, lkt, lmt] * pmf[hkt, i]
    #                         end
    #                         return payoff
    #                     end



                    end
                end
            end
        end


        return payoff
    end



    #expected payoff of answering a payment proposal

    function respond_payment(t, hkt, lkt, lmt, Pmt_array, k_W, pmf, grid=grid)


        #the minimum level of hmt necessary for m to offer a payment
        #it will be used as hmt
        ts = threshold_m(t, lkt, lmt, Pmt_array)

        #updated lower bound
        lmt = max(ts, lmt)

        return compute_payoff_pmt(t, Pmt_array, pmf, k_W, hkt, lkt, lmt, grid)

    end




    # quase mesma função que foi criada para acelerar Pkt!(), mas aqui não subtrai pkt
    function dot_U2(t, hkt, grid, U, pmf)

        #calculates the dot product between U and pmf
        payoff=0.0
        @inbounds for i in hkt:grid
            payoff+= U[t, i] * pmf[hkt, i]
        end
        return payoff
    end

    #expected payoff of answering a liquidation proposal

    function respond_liq(t, hkt, lkt, lmt, k_L, m_L, pmf, U, grid=grid)

        #payoff_reorg is the payoff of reorganizing while paying the opponent's liquidation value
        #U_{t} (\theta_{t+1}) because the skill level is of the next period
        #dot is matrix multiplication
        payoff_reorg = dot_U2(t, hkt, grid, U, pmf) - m_L(t)

        payoff_liq = k_L(t)

        return max(payoff_liq, payoff_reorg)
    end



    # @code_warntype respond_liq(t, hkt, lkt, lmt, k_L, m_L)

    #expected payoff of being called to respond
    function respond(t, hkt, lkt, lmt, Pmt_array, k_W, k_L, m_L, pmf, U, grid=grid)

        probm_liq = prob_liq_m(t, threshold_m(t, lkt, lmt, Pmt_array), lmt, pmf)


        return (1.0 - probm_liq) * respond_payment(t, hkt, lkt, lmt, Pmt_array, k_W, pmf) + probm_liq * respond_liq(t, hkt, lkt, lmt, k_L, m_L, pmf, U, grid)
    end
    
    
    function populate_pkt_manual!(t, s_W, j_W, s_L, j_L, Pst_array, Pjt_array, pmf, pmf2, share, U, Vmax, grid)

        @inbounds for l in 1:grid
            for h in 1:grid
                Pkt!(t, h, l, s_W, j_W, s_L, Pst_array, pmf, pmf2, share, U, Vmax, grid) #Pst
                Pkt!(t, h, l, j_W, s_W, j_L, Pjt_array, pmf, pmf2, share, U, Vmax, grid) #Pjt
            end
        end

    end

    function populate_wkt_manual!(t, s_W, j_W, s_L, j_L, Pst_array, Pjt_array, pmf, pmf2, share, U, grid)

        for lm in 1:grid
            for lk in 1:grid
               for hk in lk:grid #populates only if hk >= lk, saving time
                    s_W[t, hk, lk, lm] = λj * respond(t, hk, lk, lm, Pjt_array, s_W, s_L, j_L, pmf, U) + (1-λj) * propose(t, hk, lk, lm, Pst_array)[1]
                    j_W[t, hk, lk, lm] = λj * propose(t, hk, lk, lm, Pjt_array)[1] + (1-λj) * respond(t, hk, lk, lm, Pst_array, j_W, j_L, s_L, pmf, U)

                end
            end
        end
    end


    function populate_periods_manual!(tfinal, tbegin, s_W, j_W, s_L, j_L, Pst_array, Pjt_array, pmf, pmf2, share, U, Vmax, grid)

#         @time begin


            for t in tfinal:-1:tbegin

                #populating Pkt
                populate_pkt_manual!(t, s_W, j_W, s_L, j_L, Pst_array, Pjt_array, pmf, pmf2, share, U, Vmax, grid)

                #populating Wkt
                populate_wkt_manual!(t, s_W, j_W, s_L, j_L, Pst_array, Pjt_array, pmf, pmf2, share, U, grid)

            end

#         end

    end

   ### Populating the whole game
    populate_periods_manual!(T, 1, s_W, j_W, s_L, j_L, Pst_array, Pjt_array, pmf, pmf2, share, U, Vmax, grid)

    
    ### returns the arrays necessary to simulate the game
    return s_W, j_W, Pst_array, Pjt_array, Ds, Dj
    
end


In [3]:
#Vh/D, L/D, Dj/D
data1 = [0.86, 0.19, 0.8];
n1 = 49

data2 = [2.65, 0.69, 0.74];
n2 = 17

#ρ, β, c0, λj
game_parameters = [0.93, 7.856, 0.116, 0.99];

In [4]:
A2 = @spawnat aux solve_tree(data2, game_parameters)

A1 = solve_tree(data1, game_parameters);

A2 = fetch(A2);

       From worker 3:	 40.879596 seconds (142.85 M allocations: 3.216 GiB, 1.37% gc time)
49.223835 seconds (171.78 M allocations: 3.856 GiB, 1.36% gc time)


Simulation parameters

In [5]:
#θs0, θj0, μ
simulation_parameters = [35.0, 10.0, 2.387];

### Criando funções para simular o jogo

In [6]:
function simulate_moments(U, S, N, game_parameters, simulation_parameters, data1, data2, A1, A2, n1)


    Results = loop_simulations(U, S, N, game_parameters, simulation_parameters, data1, data2, A1, A2, n1)

    Moments = zeros(S, 8)

    μ = simulation_parameters[3]
    for s in 1:S

        DF = DataFrame()
        DF.payoff_s = Results[s,:,1];
        DF.payoff_j = Results[s,:,2];
        DF.out = Results[s,:,3];
        DF.t = Results[s,:,4];
        DF.observed_proposals = Results[s,:,5];
        DF.Ds = Results[s,:,6];
        DF.Dj = Results[s,:,7];


        DF.outcome = ifelse.(DF.out .==3.0, "R", "L");


        #desfazendo o deslocamento no índice de t
        DF.t = DF.t .- 1.0;

        # DF.incourt = ifelse.(DF.t .> 0.0, "incourt", "precourt");

        INCOURT = filter(DF -> DF.t .> 0.0, DF);
        PRECOURT = filter(DF -> DF.t .<= 0.0, DF);

        REORG = filter(DF -> DF.outcome .== "R", DF);#casos reorganizados dentro e fora da corte

        PRECOURT_REORG = filter(PRECOURT -> PRECOURT.outcome .== "R", PRECOURT)
        INCOURT_REORG = filter(INCOURT -> INCOURT.outcome .== "R", INCOURT)

        #cálculo dos momentos
        #1. avg log number of months between observed proposals incourt

        mm1 = INCOURT
        mm1 = log.(mm1.t .* μ ./ mm1.observed_proposals)
        mm1 = mean(mm1)

        #2. fraction reorganized given that the case went into court

        mm2 = INCOURT
        mm2 = size(filter(mm2 -> mm2.outcome .== "R", mm2),1)/ size(mm2,1)

        #3. ln duration of court cases in months

        mm3 = INCOURT
        mm3 = filter(mm3 -> mm3.t .> 0.0, mm3) #removendo os casos 0 para não poluir a média
        mm3.t = mm3.t .* μ
        mm3 = mean(log.(mm3.t)) #log here uses exp as base, so it's the same as ln

        #4. fraction of cases incourt
        mm4 = size(INCOURT,1) / size(DF, 1)

        #5. avg recovery rate for S given a REORGANIZATION in the 25% fastest cases
        q = 0.25 #quartil desejado

        mm5 = REORG

        if( size(mm5, 1) == 0) #se não tiver casos de reorganização dentro ou fora da corte, retorna zero
            mm5 = 0.0
            mm6 = 0.0
        else

            mm5.t = mm5.t .* μ #total duration
            quartil = quantile(log.(mm5.t), q)

            mm5 = filter(mm5 -> log.(mm5.t) .<= quartil, mm5)
            mm5.R_s = mm5.payoff_s ./ mm5.Ds
            mm5 = mean(mm5.R_s)

            #6. avg recovery rate for J given a REORGANIZATION in the 25% fastest cases
            mm6 = REORG
            mm6 = filter(mm6 -> log.(mm6.t) .<= quartil, mm6)
            mm6.R_j = mm6.payoff_j ./ mm6.Dj
            mm6 = mean(mm6.R_j)

        end


        #7. junior avg fraction gain given incourt REORGANIZATION

        mm7 = INCOURT_REORG
        mm7 = mean(mm7.payoff_j ./ (mm7.payoff_j .+ mm7.payoff_s )) 

        #8. total recovery rate given incourt REORGANIZATION

        mm8 = INCOURT_REORG
        mm8 = mean( mm8.payoff_s .+ mm8.payoff_j)

        Moments[s,:] .= [mm1, mm2, mm3, mm4, mm5, mm6, mm7, mm8]


        #if there are no INCOURT or no PRECOURT cases, it will return NaN
        #thus, we replace it by zero
        Moments[s,:] .= replace!(Moments[s,:], NaN=> zero(0.0))

    end





    #9. slope of the regression of log(recovery rate | incourt reorg) ~ log(duration)

    #fazendo a regressão do nono momento fora do loop para poupar tempo
    #isto é, ao invés de fazer uma regressão para cada simulação,
    #faremos uma regressão só usando todas as simulações
    df = DataFrame()

    for s in 1:S
        df_result = DataFrame(Results[s,:, 1:4])
        append!(df, df_result)
    end

    rename!(df, [:x1, :x2, :x3, :x4] .=> [:payoff_s, :payoff_j, :out, :t])



    #desfazendo o deslocamento no índice de t
    df.t = df.t .- 1.0;

    #keeping only incourt reorganizations
    df = filter(df -> df.t .> 0.0, df);
    df = filter(df -> df.out .== 3.0, df)


    x = log.(df.payoff_s .+ df.payoff_j)

    X = [ones(size(x,1)) x]

    y = log.(df.t .* μ)

    if(size(df, 1) == 0 || rank(X' * X) != size(X,2))
        #the second condition is to make sure that (X' * X) is singular before calculating mm9
        mm9 = zero(0.0)
    else
        mm9 = (X' * X) \ (X' * y)
        mm9 = mm9[2]
    end


    MM = vec(mean(Moments, dims=1))
    MM = vcat(MM, mm9)

    return MM

end



function loop_simulations(U, S, N, game_parameters, simulation_parameters, data1, data2, A1, A2, n1)


    #U is an array if dimensions S x N x 120 of fixed draws of the uniform distribution
        #120 is the upper bound of draws needed: 4 for each round (with cram dowm), maximum of 30 rounds
    #S is the number of simulations
    #N is the number of observations

    #data1 is the center of the first cluster
    #A1 is the collection of arrays from the first cluster (s_W, j_W, Pst, Pjt)
    #n1 is the number of observations in the first cluster



    #no futuro os argumentos podem ser arrays de arrays, assim ele faz o loop para cada cluster
    Results = zeros(S, N, 7)

    #últimas entradas de Results são os valores das dívidas


    for s in 1:S
        for n in 1:N
            if(n <= n1)
                Results[s,n,1:5] .= simulate_game(U[s,n,:], 1, data1, game_parameters, simulation_parameters, A1[1], A1[2], A1[3], A1[4])
                Results[s,n, end-1] = A1[5]
                Results[s,n, end] = A1[6]
            else
                Results[s,n,1:5] .= simulate_game(U[s,n,:], 1, data2, game_parameters, simulation_parameters, A2[1], A2[2], A2[3], A2[4])
                Results[s,n, end-1] = A2[5]
                Results[s,n, end] = A2[6]
            end

        end
    end

    return Results

end


function simulate_game(U_sn, draw, data, game_parameters, simulation_parameters, s_W=s_W, j_W=j_W, Pst_array=Pst_array, Pjt_array=Pjt_array, t=1, grid=100)


    #U_sn is a vector of U made for simulation s of observation n

    Vmax, L, Dj = data
    Ds = one(1.0) - Dj



    ρ, β, c0, λj = game_parameters;
    c1 = c0/30;



    hst = Int64(simulation_parameters[1])
    hjt = Int64(simulation_parameters[2])

    #assumindo que lower bounds nos períodos iniciais são as próprias habilidades iniciais
    lst = hst
    ljt = hjt

    result = zeros(5);

    #number of observed proposals
    observed_proposals = 0.0


    #recovering "T+1" from s_W 
    T_game = size(s_W,1)




    while(result[3]==0.0 && t < T_game)


        #3 draws per turn
        u1 = U_sn[draw]
        draw+=1

        u2 = U_sn[draw]
        draw+=1

        u3 = U_sn[draw]
        draw+=1


        if(u1 < λj)
            propositor = "j"

        else
            propositor = "s"

        end


        #setting the default variables according to the propositor
        Pkt_array, Cont_val, prop_index, respondent_index, m_L, hkt, lkt, hmt, lmt, lk_next, hk_next, hm_next = choose_parameters(u2, u3, propositor, hst, lst, hjt, ljt, Pst_array, Pjt_array, s_W, j_W)

        #proposal ####
        policy, payoff_prop, payment, lm_next = proposal(Pkt_array, t, hkt, lmt)

        #lm_next is the update of the adversary's lower bound
        if(lm_next>=grid+1 || policy!= 3.0) #para evitar update de lowerbound quando proponente não propõe reorg
            lm_next = lmt
        else
            lm_next = Int64(lm_next)
        end



        if(policy==3.0)

            observed_proposals += 1.0

            payoff_respondent, answer = answer_reorg(payment, Cont_val, t, hm_next, lm_next, lk_next)


            if(answer==1.0)

                result[prop_index] = payoff_prop
                result[respondent_index] = payoff_respondent
                result[3] = 3.0
                result[4] = t
                result[5] = observed_proposals

            else

                t+=1

                hst, lst, hjt, ljt = update_beliefs(propositor, hk_next, hm_next, lk_next, lm_next)
            end

        elseif(policy==2.0)

            t+=1

            hst, lst, hjt, ljt = update_beliefs(propositor, hk_next, hm_next, lk_next, lm_next)



        else      
            #(policy==1.0)

            observed_proposals += 1.0

            payoff_respondent, answer = answer_liq(m_L, t, hm_next, payoff_prop, L, Ds, Dj, Vmax, ρ, c0, c1)



            if(answer==1.0)

                result[prop_index] = payoff_prop
                result[respondent_index] = payoff_respondent
                result[3] = 1.0
                result[4] = t
                result[5] = observed_proposals

            else

                result[prop_index] = payoff_prop
                result[respondent_index] = payoff_respondent
                result[3] = 3.0
                result[4] = t
                result[5] = observed_proposals


            end
        end


        if(t==T_game)

            result[1] = s_W[T_game, 1, 1, 1]#todos os valores finais de S são iguais, então acessarei o índice 1
            result[2] = j_W[T_game, 1, 1, 1]
            result[3] = 1.0
            result[4] = t
            result[5] = max(observed_proposals, 1.0)

        end





    end


    return result
end


function choose_parameters(u2, u3, propositor, hst, lst, hjt, ljt, Pst_array, Pjt_array, s_W, j_W)

    if(propositor=="s")

        Pkt_array = Pst_array
        Cont_val = j_W
        prop_index = 1
        respondent_index = 2 #índice de j, para organizar o payoff
        m_L = j_L

        hkt = hst
        lkt = lst

        hmt = hjt
        lmt = ljt


        lk_next = hkt


        hk_next = draw_beta(u2, hkt)
        hm_next = draw_beta(u3, hmt)

    else
#             propositor=="j"
        Pkt_array = Pjt_array
        Cont_val = s_W
        prop_index = 2
        respondent_index = 1
        m_L = s_L

        hkt = hjt
        lkt = ljt

        hmt = hst
        lmt = lst


        lk_next = hkt


        hk_next = draw_beta(u2, hkt)
        hm_next = draw_beta(u3, hmt)
    end

    return Pkt_array, Cont_val, prop_index, respondent_index, m_L, hkt, lkt, hmt, lmt, lk_next, hk_next, hm_next
end


function draw_beta(u, hkt, β=game_parameters[2], grid=100)


    if(hkt == grid)

        return grid

    else

        x = 1.0 - exp(1.0/β * (log(1.0 - u) + β * log(1.0 - hkt/grid)))
        x = floor(x * grid)
#         return Int64(x * 100) #to convert in an integer
        return Int64(x)

    end
end


#cost function
function Ct(t, c0=c0, c1=c1)

    #didn't use D because is normalized to D == 1.0
    #cost at period t=0(index1) is 0
    if(t <= 1)
        return 0
    else
        return c0 + c1 * (t-1) #test to make index==1 be t==0
    end
end



#liquidation payoffs
function s_L(t, L, Ds, Dj, c0, c1)
    return min(L - Ct(t, c0, c1), Ds)
end


function j_L(t, L, Ds, Dj, c0, c1)
    return min(L - Ct(t, c0, c1) - s_L(t, L, Ds, Dj, c0, c1), Dj)
end


function proposal(Pkt_array, t, hkt, lmt)

    #pkt array será sempre do propositor, quem responder às propostas apenas olhará o seu valor de continuação

    return policy, payoff_prop, payment, lm_next = Pkt_array[t, hkt, lmt, [end,end-1, 1, 2]]

end


#maximum value of reorganization each period
function Vt(Vmax, ρ, t)

    if(t <=1)
        return Vmax
    else
        #(t-2) instead of (t-1) because we shifted the indexes in the game so as to include t==0 at index==1
        return ρ^(t-2) * Vmax
    end


end

# answer_liq
function answer_liq(m_L, t, hm_next, payoff_prop, L, Ds, Dj, Vmax, ρ, c0, c1)

    liq = (m_L(t, L, Ds, Dj, c0, c1), Vt(Vmax, ρ, t) * hm_next/100 - Ct(t, c0, c1) - payoff_prop)

    payoff_liq, answer = findmax(liq)

    return payoff_liq, answer
end

function answer_reorg(payment, Cont_val, t, hm_next, lm_next, lk_next)

    reorg_value = (payment, Cont_val[t+1, hm_next, lm_next, lk_next])

    payoff_reorg, answer = findmax(reorg_value)

    return payoff_reorg, answer
end

function update_beliefs(propositor, hk_next, hm_next, lk_next, lm_next)

    if(propositor=="s")
        hst = hk_next
        lst = lk_next

        hjt = hm_next
        ljt = lm_next

    else
        hst = hm_next
        lst = lm_next

        hjt = hk_next
        ljt = lk_next
    end


    return hst, lst, hjt, ljt
end


function error_vector(data_moments, simulated_moments)
    #two arrays with the data moments and the simulated moments
    #colocando erro ABSOLUTO, sem ponderar. Deixa que a matriz W_hat faça a ponderação
    return (simulated_moments .- data_moments)

end

update_beliefs (generic function with 1 method)

In [7]:
#unit testing
S = 100
N = 66

using DelimitedFiles

U = readdlm("U.csv", ',')
U = reshape(U, (S, N, 120));

In [8]:
# #unit testing
# s = 1
# n = 1
# @btime simulate_game(U[s,n,:], 1, data1, game_parameters, simulation_parameters, A1[1], A1[2], A1[3], A1[4])

  5.267 μs (56 allocations: 3.05 KiB)


5-element Array{Float64,1}:
 0.10211691916287892
 0.0006189333333333332
 3.0
 3.0
 1.0

In [9]:
# Results = loop_simulations(U, S, N, game_parameters, simulation_parameters, data1, data2, A1, A2, n1);

In [24]:
simulated_moments = simulate_moments(U, S, N, game_parameters, simulation_parameters, data1, data2, A1, A2, n1)

9-element Array{Float64,1}:
  2.2970684309056457
  0.6432371399477048
  2.429059230223651
  0.35469696969696973
  0.8164056029289886
  0.09586820371994731
  0.3210073191433458
  0.44908622910889695
 -0.14478619372051074

In [11]:
data_moments = [1.8192, 0.8281, 2.7704, 0.9696, 0.5652, 0.5562, 0.7835, 0.4987, -0.0962];

9-element Array{Float64,1}:
  2.2970684309056457
  0.6432371399477048
  2.429059230223651
  0.35469696969696973
  0.8164056029289886
  0.09586820371994731
  0.3210073191433458
  0.44908622910889695
 -0.14478619372051074

Calculando a derivada dos momentos em relação aos parâmetros

In [27]:
function simulate_new_moments(i, U, S, N, game_parameters, simulation_parameters, data1, data2, A1, A2, n1)
    
    
    println("parameter ",i, game_parameters, simulation_parameters)
        
    θs0, θj0, μ = simulation_parameters;
    θs0 = Int64(floor(θs0));
    θj0 = Int64(floor(θj0));

    λj = game_parameters[4];
    new_simulated_moments = simulate_moments(U, S, N, game_parameters, simulation_parameters, data1, data2, A1, A2, n1) 
    
    return new_simulated_moments
end

simulate_new_moments (generic function with 1 method)

In [40]:
function derivative_moments(U, S, N, game_parameters, simulation_parameters, data1, data2, n1, aux)

    #https://en.wikibooks.org/wiki/Introduction_to_Numerical_Methods/Numerical_Differentiation
    #center divided difference method
    #Judd (1998, page 281)

    g = game_parameters
    s = simulation_parameters

    A2 = @spawnat aux solve_tree(data2, g)
    A1 = solve_tree(data1, g)
    A2 = fetch(A2)
    
    P = size(g,1)+size(s,1)
#     M = size(simulated_moments,1)
    M = 9 #ajustar manualmente porque agora não calculamos mais simulated_moments no início
    D = zeros(M, P)
    
    #porcentagem de variação no parâmetro
    pct = 1/100
    
    
    
    for p in P:-1:1 #escolhi 5 apenas para testar, depois colocar 1 novamente

        if(p>size(g,1))

            s_plus = copy(s)
            s_minus = copy(s)
            
#             simulation_parameters = copy(estimated_simulation_parameters)
            delta = s[p-size(g,1)] * pct
            delta_denom = delta

            if( p == size(g,1)+1 || p == size(g,1)+2 ) #para ajustar caso seja habilidade inicial de S ou de J
                #as habilidades estão em números inteiros, então delta deve ser no mínimo igual a 1
                #e se for maior que um, então irá arredondar para o maior número inteiro mais próximo
                delta = ceil(delta)
                delta_denom = delta/100 #divide por 100 para que delta fique em decimais
            end
            
            s_plus[p-size(g,1)] += delta
            s_minus[p-size(g,1)] -= delta
            
            moments_plus = simulate_new_moments(p, U, S, N, g, s_plus, data1, data2, A1, A2, n1)
            moments_minus = simulate_new_moments(p, U, S, N, g, s_minus, data1, data2, A1, A2, n1)

            D[:, p] = (moments_plus - moments_minus) ./ (2 * delta_denom)
            
        else

            #se for um parâmetro de jogo, precisa recalcular o jogo

#             game_parameters = copy(estimated_game_parameters)
            delta = g[p] * pct
            
            g_plus = copy(g)
            g_plus[p] += delta
            A2 = @spawnat aux solve_tree(data2, g_plus)
            A1 = solve_tree(data1, g_plus)
            A2 = fetch(A2)
            moments_plus = simulate_new_moments(p, U, S, N, g_plus, s, data1, data2, A1, A2, n1)
            
            
            g_minus = copy(g)
            g_minus[p] -= delta
            A2 = @spawnat aux solve_tree(data2, g_minus)
            A1 = solve_tree(data1, g_minus)
            A2 = fetch(A2)
            moments_minus = simulate_new_moments(p, U, S, N, g_minus, s, data1, data2, A1, A2, n1)

            D[:, p] = (moments_plus - moments_minus) ./ (2 * delta)


        end


    end
    
    
    
    return D
end
    

derivative_moments (generic function with 1 method)

In [41]:
Derivative = derivative_moments(U, S, N, game_parameters, simulation_parameters, data1, data2, n1, aux)

       From worker 3:	 40.102482 seconds (141.59 M allocations: 3.154 GiB, 1.52% gc time)
48.322723 seconds (170.52 M allocations: 3.794 GiB, 1.28% gc time)
parameter 7[0.7, 9.0, 0.1, 0.2][10.0, 10.0, 6.12]
parameter 7[0.7, 9.0, 0.1, 0.2][10.0, 10.0, 5.88]
parameter 6[0.7, 9.0, 0.1, 0.2][10.0, 11.0, 6.0]
parameter 6[0.7, 9.0, 0.1, 0.2][10.0, 9.0, 6.0]
parameter 5[0.7, 9.0, 0.1, 0.2][11.0, 10.0, 6.0]
parameter 5[0.7, 9.0, 0.1, 0.2][9.0, 10.0, 6.0]
       From worker 3:	 40.427196 seconds (141.59 M allocations: 3.154 GiB, 1.40% gc time)
48.884874 seconds (170.52 M allocations: 3.794 GiB, 1.50% gc time)
parameter 4[0.7, 9.0, 0.1, 0.20400000000000001][10.0, 10.0, 6.0]
       From worker 3:	 39.598990 seconds (141.59 M allocations: 3.154 GiB, 1.55% gc time)
47.749492 seconds (170.52 M allocations: 3.794 GiB, 1.48% gc time)
parameter 4[0.7, 9.0, 0.1, 0.196][10.0, 10.0, 6.0]
       From worker 3:	 40.365061 seconds (141.59 M allocations: 3.154 GiB, 1.42% gc time)
48.406665 seconds (170.52 M a

9×7 Array{Float64,2}:
  0.525849    0.0506431   -0.534041  …  -4.03218   -11.1104    0.166689
  0.380352   -0.047033     0.617084      3.01257     1.00194   0.0
  1.72976     0.0311954   -0.461792     -3.05525    -0.199404  0.166689
  0.0         0.0          0.0           0.0        -2.31061   0.0
 -0.602778    0.0757364    2.45504       3.34675    -4.77304   0.0
 -0.0237841   0.00410703   0.199145  …   0.404928   -0.917332  0.0
  0.325851   -0.0159368    0.403953     -0.34321     1.10477   0.0
  0.140339   -0.00868901  -1.39042       0.544611    2.1738    0.0
 -0.109714    0.0157945    1.34675      -0.10604     1.69542   9.92262e-14

In [42]:
Derivative[:,2]

9-element Array{Float64,1}:
  0.050643097422890346
 -0.047033047515496196
  0.031195363846171623
  0.0
  0.07573635683948637
  0.004107034853290693
 -0.015936762569750747
 -0.008689006503701895
  0.01579446386155996

In [ ]:
writedlm(string("D_game_1pct", ".csv"),   Derivative, ',')